In [2]:
import cv2
import numpy as np
import glob

# Define the ChArUco board parameters
squaresX = 7  # Number of chessboard squares in X direction
squaresY = 5  # Number of chessboard squares in Y direction
squareLength = 0.03  # Length of a square in meters
markerLength = 0.015  # Length of an ArUco marker in meters

# Create the ChArUco board
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)

# Arrays to store the detected corners and ids
all_corners = []
all_ids = []
image_sizes = []

# Read calibration images
images = glob.glob('calibration_images/*.jpg')

# Debugging: Print the number of images found
print(f"Found {len(images)} images for calibration.")

for fname in images:
    img = cv2.imread(fname)
    if img is None:
        print(f"Error loading image {fname}")
        continue
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect ArUco markers in the image
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict)
    
    if ids is not None:
        # Refine the detection and detect the Charuco corners
        ret, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board)
        if ret > 4:  # Ensure at least 4 corners are detected
            all_corners.append(charuco_corners)
            all_ids.append(charuco_ids)
            image_sizes.append(gray.shape[::-1])

            # Draw and display the corners (optional)
            img = cv2.aruco.drawDetectedCornersCharuco(img, charuco_corners, charuco_ids)
            cv2.imshow('img', img)
            cv2.waitKey(500)
        else:
            print(f"Not enough corners detected in image {fname}")
    else:
        print(f"No markers detected in image {fname}")

cv2.destroyAllWindows()

# Calibrate the camera using the detected corners and ids
if len(all_corners) > 0:
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(
        charucoCorners=all_corners,
        charucoIds=all_ids,
        board=board,
        imageSize=image_sizes[0],  # Use the size of the first image
        cameraMatrix=None,
        distCoeffs=None
    )

    # Print calibration results
    if ret:
        print("Calibration was successful")
        print("Camera matrix:\n", camera_matrix)
        print("Distortion coefficients:\n", dist_coeffs)

        # Save the calibration results
        np.savez('finalcalibration_results.npz', camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)
        print("Calibration results saved.")

    else:
        print("Calibration failed. Please check your images and try again.")
else:
    print("Not enough corners for calibration. Please check your images and try again.")




Found 30 images for calibration.
Not enough corners detected in image calibration_images/calibration_img12.jpg
Not enough corners detected in image calibration_images/calibration_img10.jpg
No markers detected in image calibration_images/calibration_img29.jpg
Not enough corners detected in image calibration_images/calibration_img16.jpg
Not enough corners detected in image calibration_images/calibration_img26.jpg
No markers detected in image calibration_images/calibration_img19.jpg
Not enough corners detected in image calibration_images/calibration_img18.jpg
No markers detected in image calibration_images/calibration_img30.jpg
No markers detected in image calibration_images/calibration_img7.jpg
Not enough corners detected in image calibration_images/calibration_img22.jpg
Calibration was successful
Camera matrix:
 [[1.47038825e+03 0.00000000e+00 5.10685530e+02]
 [0.00000000e+00 1.47598217e+03 4.55136976e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[-4.5

In [1]:
import cv2
import os

# Create a folder to store calibration images if it doesn't exist
folder_name = 'calibration_images'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Initialize camera capture
cap = cv2.VideoCapture(0)  # 0 for default camera, change if you have multiple cameras

# Variables to count images
image_count = 0
max_images = 30  # Number of images you want to capture

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('Camera Feed - Press "c" to Capture', frame)

    # Check for key press
    key = cv2.waitKey(1) & 0xFF

    # If 'c' is pressed, save the image
    if key == ord('c'):
        image_count += 1
        image_name = f'calibration_img{image_count}.jpg'
        image_path = os.path.join(folder_name, image_name)
        cv2.imwrite(image_path, frame)
        print(f'Saved {image_name}')

    # Break the loop when required number of images is captured
    if image_count >= max_images:
        break

    # Break the loop if 'q' is pressed
    if key == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

print(f'{image_count} images captured and saved to {folder_name} folder.')


Saved calibration_img1.jpg
Saved calibration_img2.jpg
Saved calibration_img3.jpg
Saved calibration_img4.jpg
Saved calibration_img5.jpg
Saved calibration_img6.jpg
Saved calibration_img7.jpg
Saved calibration_img8.jpg
Saved calibration_img9.jpg
Saved calibration_img10.jpg
Saved calibration_img11.jpg
Saved calibration_img12.jpg
Saved calibration_img13.jpg
Saved calibration_img14.jpg
Saved calibration_img15.jpg
Saved calibration_img16.jpg
Saved calibration_img17.jpg
Saved calibration_img18.jpg
Saved calibration_img19.jpg
Saved calibration_img20.jpg
Saved calibration_img21.jpg
Saved calibration_img22.jpg
Saved calibration_img23.jpg
Saved calibration_img24.jpg
Saved calibration_img25.jpg
Saved calibration_img26.jpg
Saved calibration_img27.jpg
Saved calibration_img28.jpg
Saved calibration_img29.jpg
Saved calibration_img30.jpg
30 images captured and saved to calibration_images folder.


In [ ]:
import numpy as np
import cv2

# Load calibration data (camera_matrix and dist_coeffs) from the calibration_results.npz file
calibration_data = np.load('finalcalibration_results.npz')
mtx = calibration_data['camera_matrix']
dist = calibration_data['dist_coeffs']

# Load the image you want to undistort
image = cv2.imread('calibration_img4.jpg')  # Replace with your image path
height, width = image.shape[:2]

# Calculate the optimal new camera matrix and undistort the image
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))
undistorted_img = cv2.undistort(image, mtx, dist, None, new_camera_matrix)

# Display the original and undistorted images (optional)
cv2.imshow('Original Image', image)
cv2.imshow('Undistorted Image', undistorted_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the undistorted image (optional)
cv2.imwrite('undistorted_image1.jpg', undistorted_img)


In [ ]:
import numpy as np
import cv2

# Load calibration data (camera_matrix and dist_coeffs) from the calibration_results.npz file
calibration_data = np.load('finalcalibration_results.npz')
mtx = calibration_data['camera_matrix']
dist = calibration_data['dist_coeffs']

# Open the camera (use 0 for the default camera or the appropriate index for other cameras)
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Get the width and height of the camera frames
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Calculate the optimal new camera matrix
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Undistort the frame
    undistorted_frame = cv2.undistort(frame, mtx, dist, None, new_camera_matrix)
    
    cv2.imshow('Undistorted Frame', undistorted_frame)
    

    
    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()
